# Make data for worksheet 2b

In [1]:
import glob
import os
import iris

In [43]:
# Prep future data (2021-2050) from pp files
datadir = '/project/precis/worksheets/data'

# Make future 2021-2050 data
for runid in ['cahpa', 'cahpb']:
    ppdir = os.path.join(datadir, 'pp', runid)

    # find all the files from which to remove the rim
    flist = glob.glob(ppdir + '/*pm[mnop]*.pp')
    
    datacubes = iris.load(flist)
    tempcubelist = []
    for cube in datacubes:
        temprimcubelist = []
        # In case the data are more than 2 dimensional, grab a 2d slice
        for yx_slice in cube.slices(['grid_latitude','grid_longitude']):
            norimcube = yx_slice[8 : -8 , 8 : -8]
            temprimcubelist.append(norimcube)
        # Merge the individual 2d slices into a single cube
        trimmedcube = iris.cube.CubeList(temprimcubelist).merge_cube()
        tempcubelist.append(trimmedcube)
    
    for cube in tempcubelist:
        if cube.name() == 'air_temperature':
            cube.convert_units('celsius')
            outpath = os.path.join(datadir, 'netcdf', runid, runid + '.pm.2021_2050.tm.norim.K.nc')
            cube.remove_coord('forecast_period')
            cube.remove_coord('forecast_reference_time')
            iris.save(cube, outpath)
        if cube.name() == 'precipitation_flux':
            cube.convert_units('kg m-2 day-1')
            cube.units = 'mm day-1'
            outpath = os.path.join(datadir, 'netcdf', runid, runid + '.pm.2021_2050.pr.norim.mmday-1.nc')
            cube.remove_coord('forecast_period')
            cube.remove_coord('forecast_reference_time')
            iris.save(cube, outpath)

In [44]:
cube

<iris 'Cube' of precipitation_flux / (mm day-1) (time: 360; grid_latitude: 192; grid_longitude: 192)>